In [1]:
#include <iostream> 
#include <stdlib.h> 
#include <string>   
#include <cmath>    
#include <utility>
#include <unordered_map>
#include <iterator>
#include <list>
#include <stack>
#include <tuple>
#include <stdexcept>
#include <set>
//using namespace std;


# Estructuras!


Recordando al ejercicio del suset view....


Para poder modelar los edificios utilizamos las siguientes declaraciones:

```c++
typedef tuple<int,string> building;
typedef stack<building> tuple_list;
```

Y asi no solonos ahorrabamos tener que escribir: `stack< tuple<int,string> >` cada vez que queriamosun stack de edificios, o `tuple<int,string>` cada vez que queriamos un edificio, mas aun, nos permitia concentrarnos un poco mas en el problema en alto nivel.

Sin embargo, hay problemas que pueden surgir que esta "solucion" no contempla: que pasa si mientras resuelvo el problema, necesito meterle atributos adicionales a `building` para que sea mas rapido/corto? eso significa que ahora tengo que cambiar todo el codigo porquesu representacion interna cambio?

Veamos el siguiente ejemplo para un caso mas concreto:

imaginemos que tenemos una persona, la cual tiene unicamente dos datos: su `C.I : string`, y su año de nacimiento: `dob : int`, lo cual, puede ser perfectamente modelado como una tupla:

```c++
typedef tuple<int,string> person;
```

Ahora, imaginemos que tenemos una funcion que, dada una lista de personas retorna todas aquellas personas que tengan mas de 20 años:

```c++
void filtro1(list<person>*personas, list<person>* out){

    int current_year = getCurrentYear();

    for (person p : personas)
    {
        if (current_year - get<0>(p) >= 20 )
            out->push_front(p);
    }
}
```

Luego, como requerimiento adicional, decidimos que ademas queremos encontrar todas las personas mayores a 15 años cuyo año de nacimiento sea un palindromo:


```c++
void filtro2(list<person>*personas, list<person>* out){

    int current_year = getCurrentYear();

    for (person p : personas)
    {
        if (current_year - get<0>(p) >= 15 && is_palindrome(get<0>(p)) )
            out->push_front(p);
    }
}
```

Algo curioso de este codigo es que ambos tienen la misma estructura, y ambos repiten operaciones: cada vez que queramos obtener la edad de la persona tenemos que preguntar por el año actual y realizar la resta, por lo tanto decidimos refactorizar la estructura persona para que ademas guarde la edad:

```c++
typedef tuple<int,int,string> person;
```

Pero ahora viene el problema: cual `int` es la edad y cual es la fecha? 

Claro esta, nosotros como implementadores sabemos eso, puesto que disenamos la estructura! y siempre podemos comentarla, pero... que pasa si la edad estuviera en la primera posicion?

Partes de nuestro codigo pasarian a estar incorrectas! de hecho, la unica forma en que podriamos mantener el codigo intacto seria si anadieramos la propiedad al final... pero aun asi tendriamos que ir funcion por funcion, reemplazando todas las ocurrencias de `current_year - get<0>(p)` por `get<2>(p)`! algo que es bastante sencillo de errar.


Estos problemas casi puramente relacionados al estilo de programacion: capaz si hubiesemos podido colapsar ambas funciones en una sola funcion "maestra" no hubiese tanto trajin, pero en ese caso, cada vez que quisieramos añadir una condicion nueva, tendriamos que engordar la funcion "maestra" cada vez mas, lo cual no es para nada deseable (por algo los programas no son solo `main`!).

Por lo tanto, vamos a introducir no una manera de resolver problemas, sino una forma de "organizar" codigo para ahorrarnos todos estas redundancias, y con un poco de suerte, empezaremos a ver como esta forma de pensar puede ayudarnos a encontrar una solucion mas limpia.


# Entonces, como hacemos una persona?


Nuestro problema con `persona` se podia resumir en:

- `Persona` no es extensible, no es sencillo anadir o modificar el comportamiento de persona sin tener que buscar todas las ocurrencias y ver si esta es afectada por el cambio y como.
- Las funciones de `persona` estan "sueltas", estas pudiesen estar en cualquier parte del codigo dificultando su legibilidad, o peor aun, ocultando su funcionalidad! (es decir, pudiesen existir funciones que son puramente auxiliares que no aportan nada al usuario externo que estan ahi a plena vista, anadiendo complejidad innecesaria a la hora de combinar las piezas al trabajar en equipo).

Resolveremos la extensibilidad primero:

```c++
class Persona
{
    public:
        Persona();
        Persona(string CI, int dob);
        int     edad();
        int     dob();
        string  CI();
    private:
        int     dob;
        int     CI;

}

Persona::Persona()
{
    this->dob = -1;
    this->CI  = "";
}

Persona::Persona(string CI, int dob)
{
    this->dob = dob;
    this->CI  = CI;
}

int Persona::edad()
{
    return getCurrentYear() - this->dob();
}

int Persona::dob()
{
    return this->dob;
}

int Persona::string()
{
    return this->CI;
}
```

In [2]:
int getCurrentYear()
{
    return 2022;
}

In [3]:
bool is_palindrome(int i){
    int reverse = 0;
    int p = i;
    while (p > 0)
    {
        reverse = reverse * 10;
        reverse += p % 10;
        p = p / 10;
    }

    return reverse == i;

}


In [4]:
//V1
/*
class Persona
{
    public:
        Persona(std::string CI, int dob)
        {
            this->_dob = dob;
            this->_CI  = CI;
        }
        Persona()
        {
            this->_dob = -1;
            this->_CI  = "";
        }
        
        int edad()
        {
            return getCurrentYear() - this->_dob;
        }
        int dob()
        {
            return this->_dob;
        }
        std::string  CI()
        {
            return this->_CI;
        }
    private:
        int         _dob;
        std::string _CI;

};
*/

In [5]:
// V2

class Persona
{
    public:
        Persona(std::string CI, int dob)
        {
            this->_dob = dob;
            this->_CI  = CI;
            this->_age = getCurrentYear() - this->_dob;
        }
        Persona()
        {
            this->_dob = -1;
            this->_CI  = "";
            this->_age  = -1;
        }
        
        int edad()
        {
            return this->_age;
        }
        int dob()
        {
            return this->_dob;
        }
        std::string  CI()
        {
            return this->_CI;
        }
    private:
        int         _dob;
        int         _age;
        std::string _CI;

};


In [6]:
void filtro1(std::list<Persona>*personas, std::list<Persona>* out){

    for (Persona p : *personas)
    {
        if (p.edad() >= 20 )
            out->push_front(p);
    }
}

In [7]:
void filtro2(std::list<Persona>*personas, std::list<Persona>* out){

    for (Persona p : *personas)
    {
        if (p.edad() >= 15 && is_palindrome(p.dob()) )
            out->push_front(p);
    }
}

In [8]:
std::list<Persona> personas;
std::list<Persona> f1;
std::list<Persona> f2;


Persona karen   = Persona("26.385.676",1980);
Persona unknown = Persona("0",121);
Persona pepe    = Persona("24.350.144",2002);

personas.push_front(karen);
personas.push_front(unknown);
personas.push_front(pepe);

filtro1(&personas,&f1);
filtro2(&personas,&f2);

for(Persona var : f1)
{
    std::cout << var.CI() << std::endl;
}

std::cout << std::endl << std::endl;

for(Persona var : f2)
{
    std::cout << var.CI() << std::endl;
}



26.385.676
0
24.350.144


0
24.350.144


# Estructuras Recursivas!


Muy fino todo hasta ahora, vimos que con objetos podemos modelar facilmente muchas cosas de la vida real: personas, animales, cosas... Pero que si les dijera:


<center>

<img src="https://preview.redd.it/nj02sbmhlq861.jpg?auto=webp&s=508ade8cc76dc31225748b138cc9804e0da7150d" width="500" height="500" />

</center>

veamos como podemos modelar listas:



Lista a = a -> Lista a | Null

3 -> 4 -> 5 -> NULL

3 (Lista 4 -> 5 -> NULL)
3 (4  (Lista 5 -> NULL))
3 (4  (5 NULL))



In [9]:
template<typename T>
class ListNode{
    public:
        T            value;
        ListNode<T>* next;
};

In [10]:
template <typename T>
// Single Linked List <- llista simplemente enlazada
class SLL{
    
    public:
    SLL(){
        this->head = NULL;
    }

    bool isEmpty() const{
        return this->head == NULL;
    }

    T unsafeHead() const{
        return this->head->value;
    }

    SLL<T>* tail() const{
        return new SLL(this->head->next);
    }

    void push(T x){
        /*

        3 -> 4 -> 5 -> NULL

        2

        Nodo = _ -> NULL

        Node = 2 -> NULL

        Node = 2 -> 3 -> 4 -> 5 -> NULL

        this.head = Node = 2 -> 3 -> 4 -> 5 -> NULL

        */

        ListNode<T>* newHead = new ListNode<T>;
        newHead->value = x;        
        newHead->next  = this->head;
        this->head     = newHead;
    }

    SLL<T>* operator+(SLL<T>& b){
        SLL<T>* res = new SLL<T>;

        if (this->isEmpty()){
            res = &b;
            return res;
        }
            

        res = *(this->tail()) + b;
        res->push(this->unsafeHead());

        return res;
    }

    T pop(){
        /*
        3 -> 4 -> 5 -> NULL
             ^
            head

        ret = 3
        
        */

        T v = this->unsafeHead();
        ListNode<T>* temp = this->head;
        this->head = this->head->next;
        delete temp;
        return v;
    }

    /*
    T get(int i) const {
        SLL<T>* temp = this;
        while(i != 0){
            i--;
            temp = temp->tail();
        }

        return temp->unsafeHead();

    }
    */

    T get(int i) const
    {
        ListNode<T>* temp = this->head;
        
        for (; i > 0; i--)
            temp = temp->next;

        return temp->value;
    }

    T operator[](int i) const{
        ListNode<T>* temp = this->head;
        
        for (; i > 0; i--)
            temp = temp->next;

        return temp->value;
    };

    int length() const 
    {
        ListNode<T>* temp = this->head;
        int i = 0;
        while(temp != NULL)
        {
            i++;
            temp= temp->next;
        }

        return i;
    }
   
   int length2() const
    {
       if (this->isEmpty())
       {
           return 0;
       }

        SLL<T>* temp = this->tail();
        return 1 + temp->length2();
    }

    

    void print(){
        SLL<T>* aux = this;

        while(!aux->isEmpty()){
            std::cout << aux->unsafeHead() << " -> ";
            aux = aux->tail();
        }

        std::cout << "nil" << std::endl;

        return ;
    }
    
    private:

    SLL(ListNode<T>* head){
        this->head=head;
    }
    ListNode<T>* head;
          
};


In [11]:
SLL<int> listica;

std::cout << "is empty? " << listica.isEmpty() << std::endl;


listica.push(4);
listica.push(3);
listica.push(2);
listica.push(20);

std::cout << "popping: " << listica.pop() << std::endl;

std::cout << "head: " << listica.unsafeHead() << std::endl;

// 2->3->4

std::cout << "listica.get(1): " << listica.get(1) << std::endl;
std::cout << "listica[1]: " << listica[1] << std::endl;
std::cout << "listica.length():" << listica.length() << std::endl;
std::cout << "listica.length2():" << listica.length2() << std::endl;
std::cout << "listica.isEmpty():" << listica.isEmpty() << std::endl;
std::cout << "listica[1]: " << listica[1] << std::endl;

SLL<int> lista1;
SLL<int> lista2;

for (size_t i = 1; i <= 3; i++)
{
    lista1.push(i);
}

for (size_t i = 4; i <= 6; i++)
{
    lista2.push(i);
}

SLL<int> lista3;

lista3 = lista1;

lista3.print();
lista1.print();
lista2.print();


SLL<int> lista4;

lista4 = *(lista1 + lista2);
std::cout << "lista 4: ";
lista4.print();

//lista2.print();

/*
while(!listica.isEmpty())
{
    std::cout << listica.pop() << std::endl;
}
*/

// composition over inheritance.
// herencia     <- inhritance.
// polimorfismo <- polymorphism




input_line_18:25:27: error: invalid operands to binary expression ('SLL<int>' and 'SLL<int> *')
SLL<int>* lista3 = lista1 + &lista2;
                   ~~~~~~ ^ ~~~~~~~
input_line_17:32:13: note: candidate function not viable: no known conversion from 'SLL<int> *' to 'SLL<int> &' for 1st argument; remove &
    SLL<T>* operator+(SLL<T>& b){
            ^


Interpreter Error: 